In [ ]:
from IPython import parallel as p

In [ ]:
pc = p.Client()

In [ ]:
pc.ids

In [ ]:
view = pc.load_balanced_view()

In [ ]:
%%px
import numpy as np
import pandas as pd
dates = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/timecode.pcl')
wyears = np.load('/Volumes/Users/Theo/projects/Budyko_vic/wyears.npy')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']
forcing_columns = ['P','Tmax','Tmin','W']


In [ ]:
import pandas as pd
import numpy as np
import glob

dates = pd.read_pickle('./timecode.pcl')
wyears = np.load('./wyears.npy')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']
forcing_columns = ['P','Tmax','Tmin','W']

In [ ]:
def snowmeltflux(fl,fr):
    lat = float(fl.split('_')[-2]) # extract latitude from file name
    lon = float(fl.split('_')[-1]) # extract longitude from file name

    # load the fluxes data
    flux = pd.read_table(fl, sep='\t', names=fluxes_columns)
    flux.index = pd.DatetimeIndex(dates)
    
    force = pd.read_table(fr, sep=' ', names=forcing_columns)
    force.index = pd.DatetimeIndex(dates)
    
    flux['melt'] = np.NaN # preallocate the melt column
    
    s1 = flux.SWE[0:-1].as_matrix()
    s2 = flux.SWE[1:].as_matrix()
    
    melt = s1-s2
    
    melt = np.append(0,melt) # make melt the correct size
    
    coef = (0.-force.Tmin)/(np.abs(force.Tmax-force.Tmin))
    
    coef[coef<0] = 0.
    coef[coef>1] = 1.
    
    rain = force.P.as_matrix()*(1-coef) # compute rain
    
    
    rain[melt<0] = 0. # remove any flux values when there was accumulation
    
    sflength = len(rain[rain>0])
    
    if sflength > 0.:
        sfmelt = np.nansum(rain[rain>0])/sflength
    else:
        sfmelt = 0.
        
    return lat,lon,sfmelt

In [ ]:
files = pd.read_pickle('./forcing_fluxes_filenames_lat_lon_index.df')

In [ ]:
files.head()

In [ ]:
%time snowmeltflux(files.flux[270000],files.forcing[270000])

In [ ]:
res = view.map(snowmeltflux,files.flux,files.forcing)

In [ ]:
import ProgressBar as pb

In [ ]:
prog = pb.ProgressBar(len(files.forcing))

while res.ready() == False:
    prog.animate(res.progress)

In [ ]:
res.ready()

In [ ]:
lat,lon,rain = zip(*res.result)

In [ ]:
np.savez_compressed('./rain.npz',lat=lat,lon=lon,rain=rain)